In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../데이터/titanic/train.csv')
df_kg = pd.read_csv('../데이터/titanic/test.csv')
df1 = df.copy()
df1.set_index('PassengerId', inplace=True)
df_kg.set_index('PassengerId', inplace=True)
df1.drop(['Name'], axis=1, inplace=True)
df_kg.drop(['Name'], axis=1, inplace=True)

df_kg.Cabin.astype(str)

# 결측치 처리
df1.Cabin.fillna('N', inplace=True)
df1.Embarked.fillna('S', inplace=True)
df1.Age.fillna(df1.Age.median(), inplace=True)
df_kg.Cabin.fillna('N', inplace=True)
df_kg.Fare.fillna(df1.Fare.median(), inplace=True)
df_kg.Age.fillna(df1.Age.median(), inplace=True)

df1.Cabin = df1.Cabin.apply(lambda x:x[0])
df_kg.Cabin = df_kg.Cabin.apply(lambda x:x[0])

# 인코딩
og_columns = df1.columns[(df1.dtypes=='O')|(df1.dtypes=='category')|(df1.dtypes=='bool')]
og_columns

for i in og_columns:
    globals()[f'df1_{i}_encoder'] = LabelEncoder()
    globals()[f'df1_{i}_encoder'].fit(df1[i])
    globals()[f'df_kg_{i}_encoder'] = LabelEncoder()
    globals()[f'df_kg_{i}_encoder'].fit(df_kg[i])
    df1[i] = globals()[f'df1_{i}_encoder'].transform(df1[i])
    df_kg[i] = globals()[f'df_kg_{i}_encoder'].transform(df_kg[i])

# X, y 분리
X = df1.drop('Survived', axis=1)
y = df1.Survived

# train, test 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=19)

In [8]:
gbm_clf = GradientBoostingClassifier()

gbm_results = cross_val_score(gbm_clf, X_train, y_train, scoring='accuracy', cv=10, n_jobs=4)
print('gbm 교차검증:', round(np.mean(gbm_results), 3))

gbm_clf.fit(X_train, y_train)
print('gbm 예측점수:', round(np.mean(gbm_clf.score(X_val, y_val)), 3))

gbm 교차검증: 0.831


In [35]:
from sklearn.metrics import accuracy_score
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

def gbm_objective(search_space):
    model = GradientBoostingClassifier(**search_space)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return {'loss': -accuracy, 'status': STATUS_OK}

# new search space
search_space={'loss':hp.choice('loss', ['deviance', 'exponential']),
              'max_depth':hp.choice('max_depth', range(2, 10)),
              'min_samples_split':hp.uniform('min_samples_split', 0.1, 1),
              'min_samples_leaf':hp.choice('min_samples_leaf', range(1, 30)),
              'max_features':hp.choice('max_features', [None, 'sqrt', 'log2']),
              'min_impurity_decrease':hp.uniform('min_impurity_decrease', 0.1, 1),
              'ccp_alpha':hp.uniform('ccp_alpha', 0.01, 1),
              'learning_rate':hp.choice('learning_rate', [0.1, 0.01]),
              'n_estimators':hp.choice('n_estimators', [100, 200, 300, 400, 500,1000])}

# set the hyperparam tuning algorithm
algorithm=tpe.suggest
# implement Hyperopt
best_params = fmin(fn=gbm_objective,
                   space=search_space,
                   algo=algorithm,
                   max_evals=500)

params1 = space_eval(search_space, best_params)

100%|██████████| 500/500 [01:52<00:00,  4.46trial/s, best loss: -0.8777777777777778]


In [36]:
params1

{'ccp_alpha': 0.011001332269966666,
 'learning_rate': 0.1,
 'loss': 'exponential',
 'max_depth': 8,
 'max_features': 'log2',
 'min_impurity_decrease': 0.8671820004771132,
 'min_samples_leaf': 28,
 'min_samples_split': 0.32582763423323347,
 'n_estimators': 300}

In [37]:
best_gbm_clf = GradientBoostingClassifier(**params1)
best_gbm_clf.fit(X_train, y_train)
best_gbm_clf.score(X_val, y_val)

0.8666666666666667

In [43]:
best_gbm_clf.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.011001332269966666, loss='exponential',
                           max_depth=8, max_features='log2',
                           min_impurity_decrease=0.8671820004771132,
                           min_samples_leaf=28,
                           min_samples_split=0.32582763423323347,
                           n_estimators=300)

In [44]:
best_gbm_clf.fit(X_train, y_train)
kg_upload = df_kg.copy()
kg_upload['Survived'] = best_gbm_clf.predict(df_kg)
kg_upload.Survived.to_csv('./kaggle_upload_boosting2.csv')